In [1]:
from gurobipy import *

In [2]:
m1 = Model('VRPTW')

Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-11


In [3]:
from operator import le
from unicodedata import name
from numpy import conjugate
import random
from gurobipy import *
m1 = Model('vrptw')

# 20 customer -> 1.1 minute
# 30 customers -> 5 minute    1.69 minutes
# 40                          2.45 min
# 50 customer -> 160 minute


def discreteTime(timeWindow, intervalSize):   # O(lenghtOftimeWindow/size)
    tw = []
    currtime = timeWindow[0]
    while (currtime <= timeWindow[1]):
        tw.append(currtime)
        currtime = currtime+intervalSize
    return tw

customers = 5

# timeWindows = [[20, 30], [30, 40], [30, 40]]

# timeWindows = [[170, 290], [480, 640], [130, 190], [280, 330], [450, 590], [90, 100], [260, 410], [100, 160], [140, 150], [140, 250], [110, 150], [460, 570], [200, 350], [100, 270], [500, 560], [470, 590], [120, 130], [420, 600], [500, 690], [300, 380], [140, 210], [130, 270], [120, 220], [290, 490], [340, 430], [340, 380], [240, 310], [480, 540], [220, 230], [40, 110], [290, 300], [220, 230], [500, 560], [440, 540], [370, 550], [190, 230], [500, 670], [200, 400], [170, 250], [380, 450]]

# timeMatrix = [[0, 50, 60, 30, 60, 40, 50, 100, 100, 10, 50, 70, 60, 70, 50, 10, 60, 60, 40, 50, 60, 10, 40, 40, 90, 10, 20, 40, 90, 30, 50, 70, 90, 60, 80, 50, 60, 40, 60, 20], [50, 0, 
# 60, 50, 70, 10, 50, 20, 20, 90, 70, 80, 40, 100, 70, 80, 40, 20, 40, 60, 20, 60, 70, 100, 90, 60, 10, 70, 80, 10, 20, 20, 30, 70, 70, 20, 80, 100, 10, 40], [60, 60, 0, 100, 100, 100, 
# 40, 100, 80, 10, 100, 60, 20, 80, 20, 80, 30, 90, 90, 60, 100, 40, 20, 50, 80, 60, 90, 10, 60, 20, 20, 70, 90, 10, 80, 40, 30, 20, 90, 60], [30, 50, 100, 0, 60, 70, 70, 20, 80, 40, 50, 80, 40, 60, 50, 40, 60, 10, 70, 90, 20, 70, 60, 30, 70, 20, 10, 90, 80, 100, 50, 30, 70, 80, 80, 60, 10, 60, 70, 70], [60, 70, 100, 60, 0, 30, 60, 60, 50, 70, 70, 80, 10, 30, 40, 60, 60, 60, 100, 80, 90, 60, 50, 100, 30, 10, 70, 40, 10, 90, 70, 10, 60, 80, 100, 20, 50, 50, 30, 70], [40, 10, 100, 70, 30, 0, 70, 10, 40, 30, 20, 30, 20, 90, 80, 100, 30, 50, 10, 60, 90, 40, 60, 20, 50, 20, 40, 30, 50, 70, 70, 20, 40, 70, 30, 90, 50, 30, 90, 10], [50, 50, 40, 70, 60, 70, 0, 60, 60, 100, 100, 50, 80, 100, 80, 40, 20, 10, 100, 20, 30, 10, 60, 20, 80, 10, 90, 80, 80, 50, 80, 100, 90, 70, 20, 20, 40, 80, 10, 40], [100, 20, 100, 20, 60, 10, 60, 0, 20, 80, 30, 80, 10, 90, 20, 30, 90, 90, 30, 30, 60, 100, 90, 20, 10, 10, 40, 70, 90, 20, 50, 50, 70, 100, 70, 90, 30, 10, 10, 20], [100, 20, 80, 80, 50, 40, 60, 20, 0, 80, 70, 50, 20, 100, 80, 80, 40, 70, 70, 90, 100, 90, 60, 20, 90, 60, 50, 10, 10, 60, 20, 60, 100, 
# 30, 10, 100, 10, 10, 70, 10], [10, 90, 10, 40, 70, 30, 100, 80, 80, 0, 60, 10, 30, 10, 80, 60, 50, 20, 70, 70, 80, 80, 40, 90, 20, 90, 20, 30, 60, 20, 100, 70, 70, 30, 80, 70, 10, 80, 80, 10], [50, 70, 100, 50, 70, 20, 100, 30, 70, 60, 0, 100, 80, 90, 60, 60, 10, 20, 30, 100, 90, 30, 20, 40, 90, 80, 40, 100, 50, 50, 30, 10, 70, 20, 20, 50, 10, 10, 100, 30], [70, 80, 60, 80, 80, 30, 50, 80, 50, 10, 100, 0, 80, 90, 50, 20, 20, 50, 30, 50, 70, 50, 100, 80, 10, 90, 20, 100, 70, 80, 100, 50, 40, 30, 100, 40, 100, 20, 40, 90], [60, 40, 20, 40, 10, 20, 80, 10, 20, 30, 80, 80, 0, 50, 60, 60, 50, 70, 50, 50, 10, 20, 10, 20, 100, 70, 90, 50, 70, 80, 10, 90, 30, 30, 30, 40, 70, 70, 40, 90], [70, 100, 80, 60, 30, 90, 100, 90, 100, 10, 90, 90, 50, 0, 20, 40, 30, 20, 40, 10, 80, 60, 60, 10, 10, 10, 80, 60, 60, 80, 40, 20, 60, 20, 60, 60, 40, 10, 50, 80], [50, 70, 20, 50, 40, 80, 80, 20, 80, 80, 60, 50, 60, 20, 0, 80, 50, 30, 90, 60, 10, 10, 50, 70, 50, 30, 10, 20, 100, 20, 30, 100, 70, 60, 40, 40, 30, 50, 10, 20], [10, 80, 80, 40, 60, 100, 40, 30, 80, 60, 60, 20, 60, 40, 80, 0, 90, 50, 40, 70, 90, 90, 80, 30, 10, 40, 70, 50, 40, 60, 10, 60, 10, 10, 60, 100, 20, 90, 60, 10], [60, 40, 30, 60, 60, 30, 20, 90, 40, 50, 10, 20, 50, 30, 50, 90, 0, 10, 70, 50, 50, 90, 90, 70, 40, 20, 30, 10, 20, 60, 80, 80, 90, 80, 30, 10, 60, 30, 100, 60], [60, 20, 90, 10, 60, 50, 10, 90, 70, 20, 20, 50, 70, 20, 30, 50, 10, 0, 30, 30, 70, 60, 70, 90, 50, 90, 60, 80, 20, 70, 90, 50, 60, 10, 50, 10, 50, 50, 30, 90], [40, 40, 90, 70, 100, 10, 100, 30, 70, 70, 30, 30, 50, 40, 90, 40, 70, 30, 0, 70, 50, 10, 10, 50, 90, 40, 90, 30, 30, 40, 10, 50, 60, 20, 80, 80, 20, 60, 80, 80], [50, 60, 60, 90, 80, 60, 20, 30, 90, 70, 100, 50, 50, 10, 60, 70, 50, 30, 70, 0, 40, 40, 70, 100, 70, 70, 70, 100, 100, 50, 50, 30, 90, 60, 50, 100, 100, 20, 20, 100], [60, 20, 100, 20, 90, 90, 30, 60, 100, 80, 90, 70, 10, 80, 10, 90, 50, 70, 50, 40, 0, 100, 20, 100, 40, 100, 50, 40, 80, 80, 30, 50, 90, 50, 70, 50, 20, 60, 10, 60], [10, 60, 40, 70, 60, 40, 10, 100, 90, 80, 30, 50, 20, 60, 10, 90, 90, 60, 10, 40, 100, 0, 60, 90, 90, 30, 40, 70, 60, 50, 90, 80, 80, 40, 70, 90, 70, 50, 70, 10], [40, 70, 20, 60, 50, 60, 60, 90, 60, 40, 20, 100, 10, 60, 50, 80, 90, 70, 10, 70, 20, 60, 0, 100, 40, 80, 10, 10, 60, 10, 20, 40, 40, 30, 10, 70, 50, 60, 50, 80], [40, 100, 50, 30, 100, 20, 20, 20, 20, 90, 40, 80, 20, 10, 70, 30, 70, 90, 50, 100, 100, 90, 100, 0, 20, 70, 70, 40, 20, 60, 60, 10, 60, 60, 60, 90, 100, 100, 80, 30], [90, 90, 80, 70, 30, 50, 80, 10, 90, 20, 90, 10, 100, 10, 50, 10, 40, 50, 90, 70, 40, 90, 40, 20, 0, 80, 20, 100, 50, 20, 70, 90, 20, 30, 100, 50, 20, 40, 90, 70], [10, 60, 60, 20, 10, 20, 10, 10, 60, 90, 80, 90, 70, 10, 30, 40, 20, 90, 40, 70, 100, 30, 80, 70, 80, 0, 60, 90, 100, 80, 60, 10, 80, 20, 50, 40, 90, 10, 90, 90], [20, 10, 90, 10, 70, 40, 90, 40, 50, 20, 40, 20, 90, 80, 10, 70, 30, 60, 90, 70, 50, 40, 10, 70, 20, 60, 0, 
# 60, 80, 20, 100, 90, 40, 20, 90, 30, 100, 40, 80, 40], [40, 70, 10, 90, 40, 30, 80, 70, 10, 30, 100, 100, 50, 60, 20, 50, 10, 80, 30, 100, 40, 70, 10, 40, 100, 90, 60, 0, 10, 90, 60, 
# 100, 30, 100, 60, 70, 90, 100, 40, 20], [90, 80, 60, 80, 10, 50, 80, 90, 10, 60, 50, 70, 70, 60, 100, 40, 20, 20, 30, 100, 80, 60, 60, 20, 50, 100, 80, 10, 0, 30, 20, 40, 90, 80, 60, 
# 40, 10, 60, 100, 50], [30, 10, 20, 100, 90, 70, 50, 20, 60, 20, 50, 80, 80, 80, 20, 60, 60, 70, 40, 50, 80, 50, 10, 60, 20, 80, 20, 90, 30, 0, 40, 10, 30, 90, 60, 50, 40, 10, 20, 50], [50, 20, 20, 50, 70, 70, 80, 50, 20, 100, 30, 100, 10, 40, 30, 10, 80, 90, 10, 50, 30, 90, 20, 60, 70, 60, 100, 60, 20, 40, 0, 100, 70, 60, 70, 30, 70, 10, 40, 30], [70, 20, 70, 30, 
# 10, 20, 100, 50, 60, 70, 10, 50, 90, 20, 100, 60, 80, 50, 50, 30, 50, 80, 40, 10, 90, 10, 90, 100, 40, 10, 100, 0, 70, 30, 20, 40, 90, 80, 90, 50], [90, 30, 90, 70, 60, 40, 90, 70, 100, 70, 70, 40, 30, 60, 70, 10, 90, 60, 60, 90, 90, 80, 40, 60, 20, 80, 40, 30, 90, 30, 70, 70, 0, 40, 90, 70, 100, 70, 70, 80], [60, 70, 10, 80, 80, 70, 70, 100, 30, 30, 20, 30, 30, 20, 60, 10, 80, 10, 20, 60, 50, 40, 30, 60, 30, 20, 20, 100, 80, 90, 60, 30, 40, 0, 30, 60, 60, 80, 100, 20], [80, 70, 80, 80, 100, 30, 20, 70, 10, 80, 20, 100, 30, 60, 40, 60, 30, 50, 80, 50, 70, 70, 10, 60, 100, 50, 90, 60, 60, 60, 70, 20, 90, 30, 0, 90, 20, 30, 50, 40], [50, 20, 40, 60, 20, 90, 20, 90, 100, 70, 50, 40, 40, 60, 40, 100, 10, 10, 80, 100, 50, 90, 70, 90, 50, 40, 30, 70, 40, 50, 30, 40, 70, 60, 90, 0, 60, 70, 80, 20], [60, 80, 30, 10, 50, 50, 40, 30, 10, 10, 10, 100, 70, 40, 30, 20, 60, 50, 20, 100, 20, 70, 50, 100, 20, 90, 100, 90, 10, 40, 70, 90, 100, 60, 20, 60, 0, 100, 30, 100], [40, 100, 20, 60, 50, 30, 80, 10, 10, 80, 10, 20, 70, 10, 50, 90, 30, 50, 60, 20, 60, 50, 60, 100, 40, 10, 40, 100, 60, 10, 10, 80, 70, 80, 30, 70, 100, 0, 30, 70], [60, 10, 90, 70, 30, 90, 10, 10, 70, 80, 100, 40, 40, 50, 10, 60, 100, 30, 80, 20, 10, 70, 50, 80, 90, 90, 80, 40, 100, 20, 40, 90, 70, 100, 50, 
# 80, 30, 30, 0, 90], [20, 40, 60, 70, 70, 10, 40, 20, 10, 10, 30, 90, 90, 80, 20, 10, 60, 90, 80, 100, 60, 10, 80, 30, 70, 90, 40, 20, 50, 50, 30, 50, 80, 20, 40, 20, 100, 70, 90, 0]] 

# customers = len(timeWindows)

timeWindows = []
for i in range(0,customers):
    st = random.randint(1,2)*10
    et = random.randint(1,3)*10 + st
    tw = [st,et]
    timeWindows.append(tw)

print("time Windows ", timeWindows)
print()

discreteTimeWindows = []
for i in timeWindows:
    discreteTimeWindows.append(discreteTime(i, 10))

print("discreteTimeWindows", discreteTimeWindows)
print()

# starting_time = '6:00am'

# timeMatrix = [[0, 30, 10], [30, 0, 10], [10, 10, 0]]

timeMatrix = []
for i in range(0,customers):
    timeMatrix.append([0]*customers)
for i in range(0,customers):
    for j in range(0,customers):
        if(i==j):
            timeMatrix[i][j] = 0
        elif (i<j):
            timeMatrix[i][j] = random.randint(1,10)*10
            timeMatrix[j][i] = timeMatrix[i][j]

print("timeMatrix", timeMatrix)
print()


count = 0
nodeCount = []  # assigns a number to node using count and stores its value

for i in range(0, len(discreteTimeWindows)):
    c = []
    for j in range(0, len(discreteTimeWindows[i])):
        c.append(count+1)
        count += 1
    nodeCount.append(c)

print("Node Count ", nodeCount)
print()


edgeFlows = {}
edgeNumber = 0
x = [] 
edges = {}
edgesList = []  # stores edge number going out of ith node
edgesList.append([])  # for s
# solver = pywraplp.Solver.CreateSolver('GLOP')
# GRAPH CONSTRUCTION

# out of s to starting of each customer
e = []
for i in range(0, len(nodeCount)):
#     x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
    x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
#     m1.update()
    edges['x[%i]' % edgeNumber] = (0, nodeCount[i][0])
    edgeFlows[edgeNumber] = (0, nodeCount[i][0])
    e.append(edgeNumber)
    edgeNumber += 1
edgesList[0] = e

for i in range(0, len(nodeCount)):
    for j in range(0, len(nodeCount[i])-1):
        e = []
#         x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
#         m1.update()
        edges['x[%i]' % edgeNumber] = (nodeCount[i][j], nodeCount[i][j+1])
        edgeFlows[edgeNumber] = (nodeCount[i][j], nodeCount[i][j+1])
        # add edge number in edge list
        e.append(edgeNumber)
        edgesList.append(e)
        edgeNumber += 1
    edgesList.append([])
print("Edges List:", edgesList)

    
for i in range(len(discreteTimeWindows)):
    tu = nodeCount[i][-1]
    start_utime = discreteTimeWindows[i][0]
    end_utime = discreteTimeWindows[i][-1]

    # stores edges reachable from u
    for k in range(0, len(discreteTimeWindows)):
        sv = nodeCount[k][0]

        start_vtime = discreteTimeWindows[k][0]
        end_vtime = discreteTimeWindows[k][-1]


        if(i != k and start_utime + timeMatrix[i][k] > end_vtime):
            print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)

              # not reachable

        elif(i != k and start_vtime > end_utime + timeMatrix[i][k]):
            # edge bw (u,tu) -> (v,sv)
            print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)
            e = []
#             x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
            
            edges['x[%i]' % edgeNumber] = (tu, sv)  # u -> v reachable
            edgeFlows[edgeNumber] = (tu, sv)
            print(tu, "->", sv)
            e.append(edgeNumber)
            edgesList[tu].extend(e)
            edgeNumber += 1


        elif(i != k and start_vtime == end_utime + timeMatrix[i][k]):
            print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)
            e = []
#             x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
            

            edges['x[%i]' % edgeNumber] = (tu, sv)  # u -> v reachable
            print(tu, "->", sv)
            edgeFlows[edgeNumber] = (tu, sv)
            e.append(edgeNumber)
            edgesList[tu].extend(e)
            edgeNumber += 1


        elif (i != k and start_vtime < end_utime + timeMatrix[i][k]):
           
            firstReachableNode_index = 0  # first such that node >= sv
            for j in range(0, len(discreteTimeWindows[i])):
                if(timeMatrix[i][k] + discreteTimeWindows[i][j] >= start_vtime):
                    firstReachableNode_index = j
                    break
            # now starting from firstReachableNode make edges until feasible
            for l in range(firstReachableNode_index, len(discreteTimeWindows[i])):
                e = []
                reachingTime = discreteTimeWindows[i][l]+timeMatrix[i][k]
                for r in range(0, len(discreteTimeWindows[k])):
                    if(discreteTimeWindows[k][r] >= reachingTime):
                        print(edgesList[nodeCount[i][l]])
                        print(nodeCount[i][l], "->", nodeCount[k][r])
                        # edge bw these two
#                         x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
                        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
                        
                        edges['x[%i]' % edgeNumber] =(nodeCount[i][l], nodeCount[k][r])  # u -> v reachable
                        edgeFlows[edgeNumber] = (nodeCount[i][l], nodeCount[k][r])

                        e.append(edgeNumber)
                        edgesList[nodeCount[i][l]].extend(e)
                        print("hdhdh", edgesList[nodeCount[i][l]])
                        edgeNumber += 1
                        break




print('edge list b4 s t', edgesList)
print()

# into t (except s)
e = []
for i in range(0, len(nodeCount)):
#     x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
    x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
    edges['x[%i]' % edgeNumber] = (nodeCount[i][-1], -1)
    edgeFlows[edgeNumber] = (nodeCount[i][-1], -1)
    # add edge number in edge list
    edgesList[nodeCount[i][-1]].append(edgeNumber)
    edgeNumber += 1
edgesList.append([])


print("Node Count after s and t:", nodeCount)
print()

# GRAPH Construction OVER
# if I add it to  0 edge i'll get flow for 1 at the same edge  , add to 1 customer 2 customer


offset = edgeNumber

print('edge list', edgesList)
print()
print("edges", edges)
print()
print('edge number', edgeNumber)
print()

# print("edge flows:", edgeFlows)

flows = [0]*customers
for i in range(customers):
    flows[i] = edgeFlows.copy()
    keys = edgeFlows.keys()
    for j in keys:  # stores key values of edgeflows i.e 0 1 2
        # all x[] > offset are for flow variables
#         x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
       
        # remove key and replace it by edge
        t = flows[i].pop(j)
        flows[i][edgeNumber] = t
        edgeNumber += 1

# print("flows", flows)



time Windows  [[10, 40], [10, 20], [10, 30], [20, 50], [20, 40]]

discreteTimeWindows [[10, 20, 30, 40], [10, 20], [10, 20, 30], [20, 30, 40, 50], [20, 30, 40]]

timeMatrix [[0, 80, 100, 40, 60], [80, 0, 40, 70, 80], [100, 40, 0, 50, 20], [40, 70, 50, 0, 40], [60, 80, 20, 40, 0]]

Node Count  [[1, 2, 3, 4], [5, 6], [7, 8, 9], [10, 11, 12, 13], [14, 15, 16]]

Edges List: [[0, 1, 2, 3, 4], [5], [6], [7], [], [8], [], [9], [10], [], [11], [12], [13], [], [14], [15], []]
customer 0 to 1 start_utime  10  end_utime  40  start_vtime 10  end_vtime 20
customer 0 to 2 start_utime  10  end_utime  40  start_vtime 10  end_vtime 30
[5]
1 -> 13
hdhdh [5, 16]
customer 0 to 4 start_utime  10  end_utime  40  start_vtime 20  end_vtime 40
customer 1 to 0 start_utime  10  end_utime  20  start_vtime 10  end_vtime 40
customer 1 to 2 start_utime  10  end_utime  20  start_vtime 10  end_vtime 30
customer 1 to 3 start_utime  10  end_utime  20  start_vtime 20  end_vtime 50
customer 1 to 4 start_utime  10  end_uti

In [4]:
noc = 0
# # contraint for outgoing flow for a customer
for i in range(0, len(nodeCount)):  # choose one list from nodeCount
#     contraint = [0] * (edgeNumber)
    coeff = [] # store where coeff = 1
    for j in range(0, len(nodeCount[i])):  # se
        node = nodeCount[i][j]
        for k in range(0, len(edgesList[node])):
            e = edgesList[node][k]
            u = edges['x[%i]' % e][0]
            v = edges['x[%i]' % e][1]
                # print("u", u, "v", v)
            if(v not in nodeCount[i]):
                    # print('hello')
                    # print("u", u, "v", v)
                    coeff.append(x[e])
    
    m1.addConstr(( sum(coeff) == 1  ))
    noc+=1



incomingEdgeList = []
for i in range(0, len(edgesList)):
    incomingEdgeList.append([])  # adding lists in a list
for i in edges:
    v = edges[i][1]  # u -> v

    edge_no = int(i[2:-1])
    incomingEdgeList[v].append(edge_no)

print("incoming edge list: ", incomingEdgeList)
print()

#  constraint for incoming for a customer
for i in range(0, len(nodeCount)):
    coeff = []
    contraint = [0] * (edgeNumber)  # indicates number of edges
    for j in range(0, len(nodeCount[i])):
        node = nodeCount[i][j]
        for k in range(0, len(incomingEdgeList[node])):
            e = incomingEdgeList[node][k]
            u = edges['x[%i]' % e][0]
            v = edges['x[%i]' % e][1]
                # print("u", u, "v", v)
            if(u not in nodeCount[i]):
                    # print('hello')
                    # print("u", u, "v", v)
                    coeff.append(x[e])
    m1.addConstr(( sum(coeff) == 1  ))
    noc+=1
    
# constraint for a node incoming = outgoing
for i in range(0, len(nodeCount)):
    for j in range(0, len(nodeCount[i])):
        coeff = []
        contraint = [0] * (edgeNumber)
        node = nodeCount[i][j]
        for k in range(0, len(incomingEdgeList[node])):
            e = incomingEdgeList[node][k]
            contraint[e] = 1
            coeff.append(x[e])
        for k in range(0, len(edgesList[node])):
            e = edgesList[node][k]
            contraint[e] = -1
            coeff.append(-x[e])
        m1.addConstr(( sum(coeff) == 0 ))
        noc+=1  

incoming edge list:  [[], [0], [5], [6], [7], [1], [8], [2], [9], [10], [3], [11], [12], [13, 16], [4], [14, 17], [15, 18], [19, 20, 21, 22, 23]]



In [5]:
print('done')
# #  0<= xij - fij    f<x => 0<=x-f
for j in range(len(flows)):
    diff = offset*j+offset
    for k in flows[j].keys():
        coeff = [] 
        coeff.append(-x[k])
        coeff.append(x[k-diff])
        m1.addConstr(( sum(coeff) >= 0 ))
        
print('done')
#         # print(contraint," >= 0")

# # constraint for incoming for a particular customer = 1
# print("CONSTRAINT for incoming = 1 ")
for c in range(customers):
    addn = offset*(c+1)
    
    for i in range(0, len(nodeCount)):
        # indicates number of edges
        coeff = []
        for j in range(0, len(nodeCount[i])):
            node = nodeCount[i][j]
            for k in range(0, len(incomingEdgeList[node])):
                e = incomingEdgeList[node][k]
                u = edges['x[%i]' % e][0]
                v = edges['x[%i]' % e][1]
                # print("u", u, "v", v)
                if(u not in nodeCount[i]):
                    # print('hello')
                    # print("u", u, "v", v)
                    coeff.append(x[e+addn])
        m1.addConstr(( sum(coeff) == 1 ))
                    
                    


# # flow conservation for node

for c in range(customers):
    addn = offset*c+offset
    for i in range(0, len(nodeCount)):
        for j in range(0, len(nodeCount[i])):
            coeff = []
            node = nodeCount[i][j]
            for k in range(0, len(incomingEdgeList[node])):
                e = incomingEdgeList[node][k]
                coeff.append(x[e+addn])
            for k in range(0, len(edgesList[node])):
                e = edgesList[node][k]
                coeff.append(-x[e+addn])
            m1.addConstr(( sum(coeff) == 0 ))

done
done


In [6]:
objectivefun = []
for i in range(0, len(edgesList[0])):
    objectivefun.append(x[edgesList[0][i]])
obj = sum(objectivefun)
m1.setObjective(obj,GRB.MINIMIZE)
  
    
m1.optimize()
print("Runtime is :",m1.Runtime)
print("objective value : ",m1.objVal)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 251 rows, 144 columns and 524 nonzeros
Model fingerprint: 0xf11d3209
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 236 rows and 132 columns
Presolve time: 0.02s
Presolved: 15 rows, 12 columns, 40 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+00   1.000000e+01   0.000000e+00      0s
       1    3.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.000000000e+00
Runtime is : 0.029315948486328125
objective value :  3.0


In [7]:
for v in m1.getVars():
    print('%s: %g' % (v.varName,v.x))

x[0]: 1
x[1]: 1
x[2]: 1
x[3]: 0
x[4]: 0
x[5]: 0
x[6]: 0
x[7]: 0
x[8]: 1
x[9]: 1
x[10]: 0
x[11]: 0
x[12]: 0
x[13]: 0
x[14]: 0
x[15]: 0
x[16]: 1
x[17]: 0
x[18]: 1
x[19]: 0
x[20]: 1
x[21]: 0
x[22]: 1
x[23]: 1
x[24]: 1
x[25]: 1
x[26]: 1
x[27]: 0
x[28]: 0
x[29]: 0
x[30]: 0
x[31]: 0
x[32]: 1
x[33]: 1
x[34]: 0
x[35]: 0
x[36]: 0
x[37]: 0
x[38]: 0
x[39]: 0
x[40]: 1
x[41]: 0
x[42]: 1
x[43]: 0
x[44]: 1
x[45]: 0
x[46]: 1
x[47]: 1
x[48]: 1
x[49]: 1
x[50]: 1
x[51]: 0
x[52]: 0
x[53]: 0
x[54]: 0
x[55]: 0
x[56]: 1
x[57]: 1
x[58]: 0
x[59]: 0
x[60]: 0
x[61]: 0
x[62]: 0
x[63]: 0
x[64]: 1
x[65]: 0
x[66]: 1
x[67]: 0
x[68]: 1
x[69]: 0
x[70]: 1
x[71]: 1
x[72]: 1
x[73]: 1
x[74]: 1
x[75]: 0
x[76]: 0
x[77]: 0
x[78]: 0
x[79]: 0
x[80]: 1
x[81]: 1
x[82]: 0
x[83]: 0
x[84]: 0
x[85]: 0
x[86]: 0
x[87]: 0
x[88]: 1
x[89]: 0
x[90]: 1
x[91]: 0
x[92]: 1
x[93]: 0
x[94]: 1
x[95]: 1
x[96]: 1
x[97]: 1
x[98]: 1
x[99]: 0
x[100]: 0
x[101]: 0
x[102]: 0
x[103]: 0
x[104]: 1
x[105]: 1
x[106]: 0
x[107]: 0
x[108]: 0
x[109]: 0
x[110]: 0


In [8]:
print("Number of Constraints :",noc)
print("Runtime is :",m1.Runtime)

Number of Constraints : 26
Runtime is : 0.029315948486328125


In [9]:
ans = {}
for v in m1.getVars()[:offset]:
    if (v.x > 0):
#         print(v.varName[2:-1])
        ans[(v.varName[2:-1])]=v.x
ans

{'0': 1.0,
 '1': 1.0,
 '2': 1.0,
 '8': 1.0,
 '9': 1.0,
 '16': 1.0,
 '18': 1.0,
 '20': 1.0,
 '22': 1.0,
 '23': 1.0}

In [10]:
edges
new_edges = {}   # stores only those edges which are required
for key in edges:
    if((key[2:-1]) in ans):
        new_edges[(key[2:-1])] = edges[key]
new_edges

{'0': (0, 1),
 '1': (0, 5),
 '2': (0, 7),
 '8': (5, 6),
 '9': (7, 8),
 '16': (1, 13),
 '18': (8, 16),
 '20': (6, -1),
 '22': (13, -1),
 '23': (16, -1)}

In [74]:
def printpaths(ans,new_edges,start_node): # return array containing paths starting from a given node
    if(start_node == -1):
        return [-1]
    
    # search for node which starts with start_node
    paths = []
    
    for i in new_edges:
        if(start_node == new_edges[i][0]):
            res = [start_node]+printpaths(ans,new_edges,new_edges[i][1])
            paths.append(res)
            
    
    
        
    return paths
    
     
    
    

In [75]:
printpaths(ans,new_edges,0)

[[0, [1, [13, -1]]], [0, [5, [6, -1]]], [0, [7, [8, [16, -1]]]]]

In [51]:
r = []
s = [[1],2]

r+s

[[1], 2]